In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import pad_sequences

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import sklearn

2023-05-11 09:48:23.469810: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 09:48:23.921283: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-11 09:48:23.921305: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-11 09:48:25.291247: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
model = load_model("models_finaux/transfert/102-0.02253.h5")

2023-05-11 09:48:27.628128: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-11 09:48:27.628451: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-11 09:48:27.628546: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-05-11 09:48:27.628757: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-05-11 09:48:27.628856: W tensorflow/c

In [3]:
dateparse = lambda x: datetime.strptime(x, '%d-%m-%Y')
df2 = pd.read_csv("Crop_Yield_Data_challenge_2.csv", parse_dates=['Date of Harvest'], date_parser=dateparse)
df2

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha)
0,Chau_Phu,10.510542,105.248554,SA,T,2022-07-15,3.40,5500
1,Chau_Phu,10.509150,105.265098,SA,T,2022-07-15,2.43,6000
2,Chau_Phu,10.467721,105.192464,SA,D,2022-07-15,1.95,6400
3,Chau_Phu,10.494453,105.241281,SA,T,2022-07-15,4.30,6000
4,Chau_Phu,10.535058,105.252744,SA,D,2022-07-14,3.30,6400
...,...,...,...,...,...,...,...,...
552,Thoai_Son,10.364419,105.164984,WS,T,2022-04-12,7.80,6640
553,Thoai_Son,10.358094,105.189541,WS,T,2022-04-12,2.00,7200
554,Thoai_Son,10.368014,105.238516,WS,T,2022-04-12,6.20,7200
555,Thoai_Son,10.275419,105.234563,WS,T,2022-04-20,3.00,6400


In [4]:
borne = [
    (-0.0440750, 1.1379025),
    (-0.0215250, 1.1155175),
    (-0.1473925, 1.13914),
    (0.04604250, 1.2140225),
    (21762.0000, 56916.0),
    (293.0, 48423.0)
]
mini = np.array([
    -0.0440750,
    -0.0215250,
    -0.1473925,
    0.04604250,
    21762.0000,
    293.0,
])
multi = np.array([
    1.1819775, 
    1.1370425, 
    1.2865325, 
    1.16798, 
    35154.0, 
    48130.0
])

In [19]:
x2 = []
y2 = []
# for i in range(len(df2)):
for i in range(100):
    with open(f"inputs_lstm/{i}.npy", "rb") as f:
        arr = np.load(f)
        arr = np.transpose(arr, (0, 2, 3, 1))
        arr = (arr-mini)/multi
        x2.append(arr)
    y2.append([df2.loc[i, "Rice Yield (kg/ha)"]])

for i in range(len(df2)-100, len(df2)):
    with open(f"inputs_lstm/{i}.npy", "rb") as f:
        arr = np.load(f)
        arr = np.transpose(arr, (0, 2, 3, 1))
        arr = (arr-mini)/multi
        x2.append(arr)
    y2.append([df2.loc[i, "Rice Yield (kg/ha)"]])

In [20]:
x2 = pad_sequences(x2, dtype="float", maxlen=32)
y2 = np.array(y2)

In [21]:
y2_predicted = model.predict(x2)

7/7 [==============================] - 22s 3s/step


In [22]:
y2_adapted = y2_predicted * (8000-5200) + 5200

In [23]:
from sklearn.metrics import explained_variance_score

In [24]:
explained_variance_score(
    y2,
    y2_adapted
)

0.5424790650013673

In [25]:
y2_adapted

array([[6038.6475],
       [6203.705 ],
       [6352.3174],
       [6051.003 ],
       [6325.6055],
       [5999.6104],
       [6016.135 ],
       [6022.8135],
       [5998.4727],
       [6006.8022],
       [5986.498 ],
       [6398.9297],
       [6385.04  ],
       [6010.301 ],
       [6765.4336],
       [5988.9946],
       [6106.025 ],
       [6107.882 ],
       [6132.2314],
       [6019.402 ],
       [5990.7705],
       [6055.1953],
       [6048.601 ],
       [6053.5938],
       [6054.5005],
       [6144.8286],
       [5962.531 ],
       [6042.7373],
       [6093.4517],
       [6077.294 ],
       [6098.3237],
       [6089.1943],
       [6119.314 ],
       [6043.498 ],
       [6317.4404],
       [5900.6064],
       [5901.9688],
       [5901.9316],
       [5900.483 ],
       [5904.5166],
       [5903.675 ],
       [5901.042 ],
       [6092.216 ],
       [6401.5474],
       [6052.517 ],
       [6098.9683],
       [6075.374 ],
       [6086.5093],
       [6087.502 ],
       [6201.899 ],
